In [27]:
from datetime import datetime, timedelta,date
from py2neo import authenticate, Graph, Relationship
graph = Graph("http://localhost:7474/db/data/")
print ("connected")
from py2neo import Node, Relationship

connected


- Get a date
- Get all airports
- Create markets, rel date-market, rel airport-market

In [28]:
def merge(graph, thing):
    import threading
    t = threading.Thread(target=lambda x,y: x.merge(y), args=(graph,thing))
    t.setDaemon(True)
    t.start()

In [57]:
new_date = date(2017,1,15)

for i in range(10,20):
    print((new_date+timedelta(days=i)).strftime('%y%m%d'))

170125
170126
170127
170128
170129
170130
170131
170201
170202
170203


In [60]:
def add_markets(start_date):
    
    def create_market(airport, date):
            name = airport['iata'] + date['date']
            return Node('Market', market_id= name)
    
    date_node = graph.find_one("Date", 'date', str(start_date))
    

    if not date_node:
        date_node = Node("Date", date = start_date.strftime('%y%m%d'))
        
        merge(graph, date_node)

        airports = list(graph.find("Airport"))

        markets = [create_market(airport, date_node) for airport in airports]

        for m,a in zip(markets, airports):
            tx = graph.begin()

            tx.create(m)
            tx.create(Relationship(m,'HAS_AIRPORT',a))
            tx.create(Relationship(m,'HAS_DATE',date_node))
            tx.commit()

In [62]:
for i in range(180):
    print((new_date+timedelta(days=i)).strftime('%y%m%d'))
    
    add_markets(new_date+timedelta(days=i))

170115


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: DeprecationWarning: Graph.find_one is deprecated, use NodeSelector instead


(ebc0fd3:Date {date:"170115"})


C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: DeprecationWarning: Graph.find is deprecated, use NodeSelector instead


170116
(b976e95:Date {date:"170116"})
170117
(be26ff1:Date {date:"170117"})
170118
(bbba033:Date {date:"170118"})
170119
(a75e5ac:Date {date:"170119"})
170120
(cd43be7:Date {date:"170120"})
170121
(d246c70:Date {date:"170121"})
170122
(bb9b45d:Date {date:"170122"})
170123
(e318ad6:Date {date:"170123"})
170124
(c705287:Date {date:"170124"})
170125
(ee30c28:Date {date:"170125"})
170126
(af03982:Date {date:"170126"})
170127
(b8c0548:Date {date:"170127"})
170128
(d5f1df2:Date {date:"170128"})
170129
(f46e331:Date {date:"170129"})
170130
(a58af5c:Date {date:"170130"})
170131
(b0776bb:Date {date:"170131"})
170201
(a4dfd19:Date {date:"170201"})
170202
(d9ddd9d:Date {date:"170202"})
170203
(a3b6e11:Date {date:"170203"})
170204
(d77f095:Date {date:"170204"})
170205
(d5d3702:Date {date:"170205"})
170206
(e73e3f6:Date {date:"170206"})
170207
(c7b65ba:Date {date:"170207"})
170208
(d5bcbc9:Date {date:"170208"})
170209
(ccdbe3e:Date {date:"170209"})
170210
(f0742af:Date {date:"170210"})
170211
(bf85

In [50]:
api = 'ja373863706848344628128972526671'
from skyscanner.skyscanner import FlightsCache
flights_cache_service = FlightsCache(api)
feb = '2017-04'
outbound, destination ='TLS','DUB'

ret = flights_cache_service.get_grid_prices_by_date(
        market='IE',
        currency='EUR',
        locale='en-IE',
        originplace="-".join([outbound, 'SKY']),
        destinationplace="-".join([destination, 'SKY']),
        outbounddate=feb).parsed

for a,b in ret.items():
    print(a,b)

Currencies [{'Symbol': '€', 'DecimalSeparator': ',', 'SymbolOnLeft': False, 'ThousandsSeparator': '\xa0', 'DecimalDigits': 2, 'RoundingCoefficient': 0, 'SpaceBetweenAmountAndSymbol': True, 'Code': 'EUR'}]
Dates [[{'DateString': '2017-04-01'}, {'DateString': '2017-04-02'}, {'DateString': '2017-04-03'}, {'DateString': '2017-04-04'}, {'DateString': '2017-04-05'}, {'DateString': '2017-04-06'}, {'DateString': '2017-04-07'}, {'DateString': '2017-04-08'}, {'DateString': '2017-04-09'}, {'DateString': '2017-04-10'}, {'DateString': '2017-04-11'}, {'DateString': '2017-04-12'}, {'DateString': '2017-04-13'}, {'DateString': '2017-04-14'}, {'DateString': '2017-04-15'}, {'DateString': '2017-04-16'}, {'DateString': '2017-04-17'}, {'DateString': '2017-04-18'}, {'DateString': '2017-04-19'}, {'DateString': '2017-04-20'}, {'DateString': '2017-04-21'}, {'DateString': '2017-04-22'}, {'DateString': '2017-04-23'}, {'DateString': '2017-04-24'}, {'DateString': '2017-04-25'}, {'DateString': '2017-04-26'}, {'DateS

In [51]:
prices = [item['MinPrice'] if item else None for item in ret['Dates'][1]]
prices

[146.0,
 48.0,
 None,
 None,
 None,
 None,
 72.0,
 None,
 None,
 None,
 None,
 None,
 None,
 48.0,
 None,
 48.0,
 48.0,
 140.0,
 90.0,
 None,
 66.0,
 None,
 51.0,
 None,
 None,
 None,
 None,
 48.0,
 122.0,
 None]

In [56]:
feb = '2017-03'
outbound, destination ='DUB', 'TLS'

ret = flights_cache_service.get_grid_prices_by_date(
        market='IE',
        currency='EUR',
        locale='en-IE',
        originplace="-".join([outbound, 'SKY']),
        destinationplace="-".join([destination, 'SKY']),
        outbounddate=feb).parsed

new_prices = [item['MinPrice'] if item else None for item in ret['Dates'][1]]
new_prices

[None,
 130.0,
 125.0,
 None,
 125.0,
 114.0,
 132.0,
 115.0,
 None,
 None,
 90.0,
 None,
 None,
 None,
 None,
 104.0,
 133.0,
 133.0,
 179.0,
 126.0,
 100.0,
 125.0,
 None,
 None,
 None,
 55.0,
 None,
 None,
 None,
 None,
 None]